In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
DIVERSITY_UTILITY = 0.25
FRAC_ADMIT = 0.5


###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['key'] = df['R'].astype(str) + "_"  + df['T'].astype(str)

df_stratum_utility = df[['R','T','B_p']].groupby(['R','T']).mean().reset_index()
df_stratum_utility['stratum_utility'] = (df_stratum_utility['B_p'] + DIVERSITY_UTILITY * df_stratum_utility['R']).round(2)
df_stratum_utility['key'] = df_stratum_utility['R'].astype(str) + "_" + df_stratum_utility['T'].astype(str)

df = df.merge(df_stratum_utility[['stratum_utility','key']], on='key')
df['ml_outcomes'] = df['stratum_utility']

In [3]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_black','T_white','B_p']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_black,T_white,B_p
98627,0,20,0.00,15,20,0
99515,0,19,0.00,17,19,0
99514,0,19,0.00,18,19,0
99920,0,15,0.00,13,15,0
99513,0,19,0.00,18,19,0
...,...,...,...,...,...,...
99369,1,92,1.25,92,105,1
99370,1,92,1.25,92,105,1
99371,1,92,1.25,92,108,1
99943,1,85,1.25,85,98,1


In [4]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [5]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','ml_outcomes']].groupby(['R','T']).sum().reset_index()

In [6]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes']].groupby(['R','T']).count().reset_index()
df_count.columns = ['R','T','Count']
df_count['N'] = df_count['Count']

In [7]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [8]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,ml_outcomes,N
0,191,0,0,14,0.00,1
1,186,1,0,15,0.00,2
2,190,2,0,16,0.00,2
4,189,4,0,18,0.00,6
5,188,5,0,19,0.00,10
...,...,...,...,...,...,...
46,4,46,0,60,1088.08,1876
45,3,45,0,59,1100.26,1897
47,2,47,0,61,1114.24,1741
44,1,44,0,58,1124.04,1972


### Setup optimization problem 

In [9]:
from ortools.linear_solver import pywraplp


In [10]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [11]:
dff

,level_0,index,R,T,ml_outcomes,N
0,191,0,0,14,0.00,1
1,186,1,0,15,0.00,2
2,190,2,0,16,0.00,2
3,165,3,0,17,2.00,5
4,189,4,0,18,0.00,6
...,...,...,...,...,...,...
187,160,187,1,95,2.50,2
188,161,188,1,97,2.50,2
189,174,189,1,98,1.25,1
190,172,190,1,99,1.25,1


In [12]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [13]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [14]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total admits cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

50000


In [15]:
# Now we have one constraint
solver.NumConstraints()

1

## Solve linear program

In [16]:
solver.ABNORMAL

4

In [17]:
status = solver.Solve()


In [18]:
status

0

In [19]:
solver.OPTIMAL

0

In [20]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [21]:
df_decisions

,row_id,decision
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
187,187,1.0
188,188,1.0
189,189,1.0
190,190,1.0


In [22]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,ml_outcomes,N,row_id,decision
43,0,43,0,57,1129.52,2017,43,1.0
44,1,44,0,58,1124.04,1972,44,1.0
47,2,47,0,61,1114.24,1741,47,1.0
45,3,45,0,59,1100.26,1897,45,1.0
46,4,46,0,60,1088.08,1876,46,1.0
...,...,...,...,...,...,...,...,...
6,187,6,0,20,0.00,11,6,0.0
5,188,5,0,19,0.00,10,5,0.0
4,189,4,0,18,0.00,6,4,0.0
2,190,2,0,16,0.00,2,2,0.0


In [23]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,ml_outcomes,N,row_id,decision
105,185,105,1,11,0.25,1,105,0.0
106,183,106,1,12,0.50,2,106,0.0
0,191,0,0,14,0.00,1,0,0.0
107,184,107,1,14,0.25,1,107,0.0
108,178,108,1,15,1.00,4,108,0.0
...,...,...,...,...,...,...,...,...
100,168,100,0,114,2.00,2,100,1.0
101,164,101,0,115,2.00,2,101,1.0
102,181,102,0,116,1.00,1,102,1.0
103,180,103,0,120,1.00,1,103,1.0


In [24]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,ml_outcomes,N,row_id,decision
43,0,43,0,57,1129.52,2017,43,1.0
44,1,44,0,58,1124.04,1972,44,1.0
47,2,47,0,61,1114.24,1741,47,1.0
45,3,45,0,59,1100.26,1897,45,1.0
46,4,46,0,60,1088.08,1876,46,1.0
...,...,...,...,...,...,...,...,...
6,187,6,0,20,0.00,11,6,0.0
5,188,5,0,19,0.00,10,5,0.0
4,189,4,0,18,0.00,6,4,0.0
2,190,2,0,16,0.00,2,2,0.0


In [25]:
len(xxx_)

192

In [26]:
xxx.to_csv('./decision.csv')

In [27]:
solution = pd.read_csv('./decision.csv')

In [28]:
xxx

,level_0,index,R,T,ml_outcomes,N,row_id,decision
43,0,43,0,57,1129.52,2017,43,1.0
44,1,44,0,58,1124.04,1972,44,1.0
47,2,47,0,61,1114.24,1741,47,1.0
45,3,45,0,59,1100.26,1897,45,1.0
46,4,46,0,60,1088.08,1876,46,1.0
...,...,...,...,...,...,...,...,...
6,187,6,0,20,0.00,11,6,0.0
5,188,5,0,19,0.00,10,5,0.0
4,189,4,0,18,0.00,6,4,0.0
2,190,2,0,16,0.00,2,2,0.0


In [29]:
(xxx['ml_outcomes']*xxx['decision']).sum()

34229.79000000001

In [30]:
def maxMLOutcomes(frac_black=0.01):
    black_admits = int(K*frac_black)
    white_admits = int(K*(1-frac_black))
    
    b = df[df['R']==1].sort_values(by='ml_outcomes',ascending=False).head(n=black_admits)
    w = df[df['R']==0].sort_values(by='ml_outcomes',ascending=False).head(n=white_admits)
    
    return b['B_p'].sum() + w['B_p'].sum()    

In [31]:
def maxMLOutcomesBlack(frac_black=0.01):
    black_admits = int(K*frac_black)
    white_admits = int(K*(1-frac_black))
    
    b = df[df['R']==1].sort_values(by='ml_outcomes',ascending=False).head(n=black_admits)
    w = df[df['R']==0].sort_values(by='ml_outcomes',ascending=False).head(n=white_admits)
    
    return b['B_p'].sum()   

In [32]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)

In [33]:
len(xxx)

192

In [34]:
admit_decisions = df.merge(xxx,how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT


FRAC_BLACK_POLICY = (admit_decisions['R_y'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R_y'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision_random']).sum()


In [35]:
file = open('./lp_results.csv','w')
file.write('{}\t{}\t{}\n'.format('Policy Name','Frac Diverse Candidates','Total Graduated'))
file.write('{}\t{}\t{}\n'.format('Random',str(FRAC_RANDOM_POLICY),str(SUM_BP_RAND_POLICY)))
file.write('{}\t{}\t{}\n'.format('Maximize Utility',str(FRAC_BLACK_POLICY),str(SUM_BP_POLICY)))
file.close()


In [36]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.5000000000000001